# Introdução

É preciso instalar os módulos: Langchain e langchain-openai

                pip install langchain
                pip install langchain-openai

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key= "sk-aaaaaa-MODIFICAR", temperature = 0.9, model_name = "gpt-3.5-turbo")



Depois de instalar e inicializar o LLM de sua escolha, podemos tentar usá-lo! Vamos perguntar qual a capital de João Pessoa na Paraíba - isso é algo que não estava presente nos dados de treinamento, então não deve ter uma resposta muito boa.

In [5]:

llm.invoke("Qual a capital do estado da Paraíba no Brasil?")

AIMessage(content='A capital do estado da Paraíba no Brasil é João Pessoa.')

Também podemos orientar sua resposta com um modelo de prompt. Modelos de prompt são usados para converter a entrada bruta do usuário em uma entrada melhor para o LLM.

In [21]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an internationally renowned cybersecurity analyst and DPO, specialist in various Information Security laws and ISOs including ISO 27001, ISO 27002, ISO 27701 and Brail's data protection law, LGPD Law 13709 of 2018. Your answers always will be in Brazilian Portuguese."),
    ("user", "{input}")
])

Combinando o Prompt (Perfil) com a entrada

In [22]:
chain = prompt | llm 

In [23]:
chain.invoke({"input": "Me fale um pouco sobre os perfis descritos na lei geral de proteção de dados Brasileira: Controlador e Titular."})

AIMessage(content='A Lei Geral de Proteção de Dados (LGPD) Brasileira define dois perfis principais: o Controlador e o Titular.\n\nO Controlador é quem toma as decisões referentes ao tratamento dos dados pessoais. Ele é responsável por determinar como e por que os dados serão processados. O Controlador pode ser uma pessoa física ou jurídica, de direito público ou privado, que detém o poder de decisão sobre o tratamento dos dados.\n\nO Titular é a pessoa a quem os dados pessoais se referem. É o indivíduo que tem seus dados pessoais coletados e processados pelo Controlador. O Titular tem direitos garantidos pela LGPD, como o acesso aos seus dados, a correção de informações incorretas, a anonimização ou eliminação dos dados, entre outros.\n\nÉ importante ressaltar que o Controlador deve respeitar os direitos e garantias do Titular, conforme estabelecido pela LGPD. Ele deve contar com o consentimento do Titular para realizar o tratamento dos dados, a menos que exista uma outra base legal p

A saída de um ChatModel (e, portanto, desta cadeia) é uma mensagem. No entanto, muitas vezes é muito mais conveniente trabalhar com cordas. Vamos adicionar um analisador de saída simples para converter a mensagem de bate-papo em uma string.

In [24]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [25]:
#Adicionando isto a cadeia anterior teremos:
chain = prompt | llm | output_parser

In [26]:
chain.invoke({"input": "Me fale um pouco sobre os perfis descritos na lei geral de proteção de dados Brasileira: Controlador e Titular."})

'Na Lei Geral de Proteção de Dados (LGPD), dois perfis são fundamentais para entendermos: o Controlador e o Titular.\n\nO Controlador é a pessoa física ou jurídica, de direito público ou privado, que toma as decisões referentes ao tratamento de dados pessoais. Ele é responsável por definir como e para quais finalidades os dados serão utilizados, determinar quais dados serão coletados, como serão armazenados, protegidos e compartilhados.\n\nJá o Titular é a pessoa física a quem se referem os dados pessoais que estão sendo tratados. É o indivíduo sobre o qual os dados se referem, sendo o seu direito fundamental a proteção de sua privacidade e intimidade.\n\nO Titular possui uma série de direitos, como o direito de acesso aos seus dados, o direito de correção, atualização ou exclusão dos dados, o direito de consentir ou não com o tratamento, o direito de se opor a determinado tratamento, entre outros.\n\nÉ importante destacar que, de acordo com a LGPD, o Controlador deve garantir ao Titul

Para responder adequadamente à pergunta ("Qual a diferença entre as ISO 27001:2013 e 27001:2022"), precisamos fornecer contexto adicional ao LLM. Podemos fazer isso por recuperação. A recuperação é útil quando você tem muitos dados para passar diretamente para o LLM. Você pode então usar um retriever para buscar apenas as peças mais relevantes e passá-las.

Neste processo, vamos procurar documentos relevantes de um Retriever e depois passá-los para o prompt. Um Retriever pode ser apoiado por qualquer coisa - uma tabela SQL, internet, etc. - mas neste caso, preencheremos uma loja de vetores e a usaremos como uma retriever. Para mais informações sobre vectorstores, consulte esta documentação.

Primeiro, precisamos carregar os dados que queremos indexar. #Para fazer isso, usaremos o WebBaseLoader.

In [33]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.linkedin.com/pulse/saiba-diferenças-entre-iso-270012013-e-270012022-marcelo-creazzo/?originalSubdomain=pt")

docs = loader.load()

Em seguida, precisamos indexá-lo em uma loja de vetores. Isso requer alguns componentes, ou seja, um modelo de incorporação e avectorstore.
Certifique-se de ter o pacote `langchain_openai` instalado e as variáveis de ambiente apropriadas definidas (estas são as mesmas necessárias para o LLM).


In [34]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key= "sk-MODIFICAR")



Agora, podemos usar esse modelo de incorporação para ingerir documentos em uma loja de vetores. Usaremos uma loja de vetores local simples, FAISS, por uma questão de simplicidade.

Primeiro, precisamos instalar os pacotes necessários para isso:

                pip install faiss-cpu

In [35]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Agora que temos esses dados indexados em um armazenamento de vetores, criaremos uma cadeia de recuperação. Esta cadeia levará uma pergunta recebida, procurará documentos relevantes, depois passará esses documentos junto com a pergunta original para um LLM e pedirá que ele responda à pergunta original.

Primeiro, vamos configurar a cadeia que pega uma pergunta e os documentos recuperados e gera uma resposta

In [36]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

Se quiséssemos, poderíamos fazer isso nós mesmos, passando documentos diretamente:

                from langchain_core.documents import Document

                    document_chain.invoke({
                    "input": "how can langsmith help with testing?",
                    "context": [Document(page_content="langsmith can let you visualize test results")]
                })
No entanto, queremos que os documentos venham primeiro do retriever que acabamos de configurar. Dessa forma, para uma determinada pergunta, podemos usar o retriever para selecionar dinamicamente os documentos mais relevantes e passá-los.



In [37]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

Agora podemos invocar essa cadeia. Isso retorna um dicionário - a resposta do LLM está na chave de answer

In [38]:
response = retrieval_chain.invoke({"input": "Qual a diferença entre a ISO 27001:2013 e a ISO 27001:2022?"})
print(response["answer"])


A diferença entre a ISO 27001:2013 e a ISO 27001:2022 está relacionada às alterações nos controles do Anexo, que foram reagrupados, renomeados e adicionados. Além disso, ocorreram mudanças nas cláusulas 4-10, que foram atualizadas para alinhar o texto com outras normas de gerenciamento ISO. As mudanças incluem orientações adicionais sobre objetivos de segurança da informação, cibersegurança, planejamento de mudanças e controle operacional, entre outras. A estrutura dos controles do Anexo A também foi modificada, com a introdução de novos controles, a mesclagem e renomeação de outros, e a remoção de três controles. As organizações certificadas na ISO 27001:2013 precisam se atualizar para a nova versão até 2025.


In [39]:
response = retrieval_chain.invoke({"input": "Qual assunto tratamos na questão anterior?"})
print(response["answer"])

Na questão anterior, o assunto tratado foi a nova versão da norma ISO/IEC 27001 lançada em 2022, as principais mudanças e atualizações feitas na norma, incluindo as mudanças nos controles do Anexo A.


# Cadeia de Recuperação de Conversação

A cadeia que criamos até agora só pode responder a perguntas únicas. Um dos principais tipos de aplicativos LLM que as pessoas estão construindo são os bots de bate-papo. Então, como transformamos essa cadeia em uma que possa responder a perguntas de acompanhamento?

Ainda podemos usar a função create_retrieval_chain, mas precisamos mudar duas coisas:

1. O método de recuperação agora não deve apenas funcionar na entrada mais recente, mas deve levar em conta toda a história.
2. A cadeia final de LLM também deve levar em conta toda a história

## Atualizando a Recuperação

Para atualizar a recuperação, criaremos uma nova cadeia. Esta cadeia receberá a entrada mais recente **´(input)´** e o histórico de conversa **´(chat_history)´** e usará um LLM para gerar uma consulta de pesquisa.

In [40]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get"+
     " information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

Podemos testar isso passando em um caso em que o usuário está fazendo uma pergunta de acompanhamento.

In [42]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Você poderia gerar um formulário para identificar se uma empresa está em conformidade com a ISO 27001?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Gere o Formulário"
})

[Document(page_content='A Organização Internacional de Normalização (ISO) lançou uma nova versão da norma ISO/IEC 27001 em outubro de 2022. Esse padrão é utilizado em todo o mundo para estabelecer requisitos de controle e criar um Sistema de Gerenciamento de Segurança da Informação (ISMS). A ISO 27001:2022 é uma atualização moderada da versão anterior, a ISO 27001:2013, e a maioria das alterações estão relacionadas aos controles do Anexo, que foram reagrupados, renomeados e adicionados. As principais mudanças que ocorreram nas cláusulas 4-10, que foram atualizadas para alinhar o texto com outras normas de gerenciamento ISO. As alterações incluem orientações adicionais sobre objetivos de segurança da informação, cibersegurança, planejamento de mudanças e controle operacional, entre outras. Além disso, foram feitas algumas pequenas mudanças na cláusula de papéis, responsabilidades e autoridades organizacionais, comunicação, auditoria interna, revisão da administração e melhoria.As grande

Você deve ver que isso retorna documentos sobre as ISOs 27001:2013 e ISI 27001:2022. Isso ocorre porque o LLM gerou uma nova consulta, combinando o histórico de bate-papo com a pergunta de acompanhamento.

Agora que temos esse novo retriever, podemos criar uma nova cadeia para continuar a conversa com esses documentos recuperados em mente.

In [43]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

Agora veremos o resultado da busca:

In [44]:
chat_history = [HumanMessage(content="Você poderia gerar um formulário para identificar se uma empresa está em conformidade com a ISO 27001?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Escreva as questões: "
})

{'chat_history': [HumanMessage(content='Você poderia gerar um formulário para identificar se uma empresa está em conformidade com a ISO 27001?'),
  AIMessage(content='Yes!')],
 'input': 'Escreve as questões: ',
 'context': [Document(page_content='A Organização Internacional de Normalização (ISO) lançou uma nova versão da norma ISO/IEC 27001 em outubro de 2022. Esse padrão é utilizado em todo o mundo para estabelecer requisitos de controle e criar um Sistema de Gerenciamento de Segurança da Informação (ISMS). A ISO 27001:2022 é uma atualização moderada da versão anterior, a ISO 27001:2013, e a maioria das alterações estão relacionadas aos controles do Anexo, que foram reagrupados, renomeados e adicionados. As principais mudanças que ocorreram nas cláusulas 4-10, que foram atualizadas para alinhar o texto com outras normas de gerenciamento ISO. As alterações incluem orientações adicionais sobre objetivos de segurança da informação, cibersegurança, planejamento de mudanças e controle oper